# Modeling data

Modeling data helps us describe the data we have. A simple linear regression is a model. Distributions of our data are a model. Curve fitting is modeling. We have used these models in both the current clamp and mini analysis chapters but we will go more in-depth here. Modeling can be divided into what, how and why models (see: https://compneuro.neuromatch.io/tutorials/intro.html for more). We will primarily focus on what models. I rarely see what models discussed in patch clamp ephys analysis, however I believe they are needed to analyze our data well.

## Distributions
Distributions of data are one of the first things you should look at especially if you have large sample sizes. Think mini amplitude, IEI, and rise rate. Most the statistical analyses we do make assumptions about the distribution of our data yet rarely do papers show distributions of their data. Even something as simple as the mean generally assumes your data follows a normal distribution. Most other distribution contain the equivalent of the mean but the value is quiet different from the mean. When taking the mean of non-gaussian distributions you are not get the true central tendency of your data and may be measuring the effect of outliers rather than a true shift in the distribution of data.

First we will start by looking at some of the data we previously collected.